In [1]:
import json
import uuid
import random
from datetime import datetime, timedelta
import os 
import sys 
import pandas as pd
import numpy as np
import requests
import zipfile
import xlsxwriter
import  psycopg2
from sqlalchemy import create_engine, inspect
import sqlalchemy as sa
import openpyxl
import argparse 

In [2]:
class PostgresCredentials:
    postgres_credentials = {
        'postgres_username' : 'postgres',
        'postgres_password' : 'radek',
        'postgres_host' : 'localhost',
        'postgres_port' : '5432',
    }

In [3]:
class PostgresConnection(PostgresCredentials):
    def __init__(self, username, password,host,port):
        self.username = username
        self.password = password
        self.host = host
        self.port = port
        
    def get_connectin_database(self):
        conn = psycopg2.connect(
            dbname="postgres",
            user=self.username,
            password=self.password,
            host=self.host,
            port=self.port
        )
        cur = conn.cursor()
        return conn,cur
    
    def get_engine(self):
        engine = create_engine(f'postgresql://{self.username}:{self.password}@{self.host}:{self.port}/')
        return engine
    
    def get_ingine_data_base(self,new_database):
        engine = create_engine(f'postgresql://{self.username}:{self.password}@{self.host}:{self.port}/{new_database}')
        return engine
    
    def commit_connection(self, connection):
        connection.commit()
        
    def close_connection(self,connection, cursor):
        cursor.close()
        connection.close()

In [2]:
folder_epidemic = r'/home/r/git_projekty/dataProjects/MalaysiaCovid/malaysia_covid/epidemic'
folder_vacination = r'/home/r/git_projekty/dataProjects/MalaysiaCovid/malaysia_covid/vaccination'


In [3]:
def show_content(folder ):
    result = os.listdir(folder)
    return result 

In [4]:
a_folder_epidemic = show_content(folder_epidemic)
# a_folder_vacination = 

In [5]:
print( a_folder_epidemic )

['cases_age.csv', 'hospital.csv', 'tests_malaysia.csv', 'linelist', 'deaths_age.csv', 'README.md', 'icu.csv', 'deaths_malaysia.csv', 'cases_state.csv', 'deaths_state.csv', 'cases_malaysia.csv', 'pkrc.csv', 'tests_state.csv', 'clusters.csv']


In [12]:
file_wyscigi = r'Wyscigi.txt'
def file_text_to_df(file_name):
    df = pd.read_table(file_name,sep=';')#,header=True)
    return df

In [13]:
new_dict = {}
for file in a_folder_epidemic:
    
    if file.endswith('.csv'):
        print( file )   
        file_name = file.split('.csv')[0]
        print( file_name )
        df =  pd.read_csv( os.path.join(folder_epidemic,file), sep=',')
        print( df.head(3))

cases_age.csv
cases_age
  week     state  abs_0_4  abs_5_11  abs_12_17  abs_18_29  abs_30_39  \
0  w24  Malaysia       35        33         24        365        417   
1  w23  Malaysia       40        34         32        367        456   
2  w22  Malaysia       39        40         37        481        488   

   abs_40_49  abs_50_59  abs_60_69  ...  capita_0_4  capita_5_11  \
0        190        131         86  ...        1.42         0.91   
1        218        149         88  ...        1.62         0.94   
2        300        160         96  ...        1.58         1.10   

   capita_12_17  capita_18_29  capita_30_39  capita_40_49  capita_50_59  \
0          0.76          4.98          7.62          4.95          4.20   
1          1.02          5.01          8.33          5.67          4.77   
2          1.18          6.57          8.91          7.81          5.13   

   capita_60_69  capita_70_79  capita_80_+  
0          3.94          6.63        22.41  
1          4.04        

In [5]:
wyscigi = file_text_to_df( file_wyscigi)

In [2]:
file_excel = r'unavailable_sessions_.xlsx'


In [3]:
excel_file = pd.ExcelFile(file_excel )
print( dir(excel_file))


['ODFReader', 'OpenpyxlReader', 'PyxlsbReader', 'XlrdReader', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__fspath__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_engines', '_io', '_reader', 'book', 'close', 'engine', 'io', 'parse', 'sheet_names', 'storage_options']


In [4]:
sheets_name = excel_file.sheet_names
print( sheets_name )

['Search by session ID', 'Search by session path', 'Unavailable sessions', 'SMS update history', 'FMS removal history', 'Marker update history', 'Image metadata removal history', 'Thumbnails removal history', 'Erasure coding volumes']


In [6]:
sheets_names_dict = {}
for name in sheets_name:
    name_temp = name.lower()
    name_temp = name_temp.replace(" ","_")
    sheets_names_dict[name_temp] = name
print( sheets_names_dict)
    

{'search_by_session_id': 'Search by session ID', 'search_by_session_path': 'Search by session path', 'unavailable_sessions': 'Unavailable sessions', 'sms_update_history': 'SMS update history', 'fms_removal_history': 'FMS removal history', 'marker_update_history': 'Marker update history', 'image_metadata_removal_history': 'Image metadata removal history', 'thumbnails_removal_history': 'Thumbnails removal history', 'erasure_coding_volumes': 'Erasure coding volumes'}


In [8]:
print( sheets_names_dict)
list_sheets_to_panda = ['unavailable_sessions', 'sms_update_history', 'fms_removal_history', 'marker_update_history', 'image_metadata_removal_history', 'thumbnails_removal_history' , 'erasure_coding_volumes']
print( list_sheets_to_panda )

{'search_by_session_id': 'Search by session ID', 'search_by_session_path': 'Search by session path', 'unavailable_sessions': 'Unavailable sessions', 'sms_update_history': 'SMS update history', 'fms_removal_history': 'FMS removal history', 'marker_update_history': 'Marker update history', 'image_metadata_removal_history': 'Image metadata removal history', 'thumbnails_removal_history': 'Thumbnails removal history', 'erasure_coding_volumes': 'Erasure coding volumes'}
['unavailable_sessions', 'sms_update_history', 'fms_removal_history', 'marker_update_history', 'image_metadata_removal_history', 'thumbnails_removal_history', 'erasure_coding_volumes']


In [37]:
# Replace 'your_excel_file.xlsx' with the path to your Excel file.
excel_file = 'your_excel_file.xlsx'
def read_excel( excel_file, sheet_name  ):
    # Open the Excel file.
    workbook = openpyxl.load_workbook(excel_file, data_only=False)

    # Select the desired sheet. Replace 'Sheet1' with the name of the sheet you want to read.
    sheet = workbook[sheet_name]

    # Iterate through the rows in the sheet and read data line by line.
    for row in sheet.iter_rows(values_only=False):
        for cell in row:
            if cell.hyperlink:
                # If the cell contains a hyperlink, print the hyperlink address.
                print(cell.hyperlink.target)
            else:
                # If the cell doesn't contain a hyperlink, print its value or formula.
                print(cell.value, end='\t')
        print()  # Move to the next line for the next row.

    # Close the Excel file.
    workbook.close()
        

In [38]:
read_excel( 'unavailable_sessions_.xlsx','SMS update history')

session_id	status_updated	chg	link	
000d9a1b-28e9-45ea-abc5-0187843b1417	true	CHG0788076	=HYPERLINK("https://csc.service-now.com/nav_to.do?uri=change_request.do?sysparm_query=number="&session_metadata[[#This Row],[chg]],"Link")	
0014f318-5109-4e14-a055-018409a90969	true	CHG0788076	=HYPERLINK("https://csc.service-now.com/nav_to.do?uri=change_request.do?sysparm_query=number="&session_metadata[[#This Row],[chg]],"Link")	
00234e2b-fdd7-4dc4-a368-0182d9f3a28a	true	CHG0788076	=HYPERLINK("https://csc.service-now.com/nav_to.do?uri=change_request.do?sysparm_query=number="&session_metadata[[#This Row],[chg]],"Link")	
00253160-d55b-4c3e-ad75-018718d80756	true	CHG0788076	=HYPERLINK("https://csc.service-now.com/nav_to.do?uri=change_request.do?sysparm_query=number="&session_metadata[[#This Row],[chg]],"Link")	
0030c011-9ae5-4636-aedc-0187a403e759	true	CHG0788076	=HYPERLINK("https://csc.service-now.com/nav_to.do?uri=change_request.do?sysparm_query=number="&session_metadata[[#This Row],[chg]],"Link")	

In [9]:
def create_dataframe_from_excel_sheet(file_name, name_sheet):
    result = pd.read_excel(file_name, sheet_name=name_sheet)
    return result



In [10]:
a1 = create_dataframe_from_excel_sheet(file_excel, 'Erasure coding volumes')

In [16]:
a1_columns =  a1.columns 
print( a1_columns , type(a1_columns))
a1_columns_list = list( a1_columns )
print( a1_columns_list , type(a1_columns_list))
print( a1.shape)
print( a1.info(verbose=True) )


Index(['path', 'vin', 'session_id', 'recording_date', 'volume_type'], dtype='object') <class 'pandas.core.indexes.base.Index'>
['path', 'vin', 'session_id', 'recording_date', 'volume_type'] <class 'list'>
(19098, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19098 entries, 0 to 19097
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   path            19098 non-null  object        
 1   vin             19098 non-null  object        
 2   session_id      19098 non-null  object        
 3   recording_date  19098 non-null  datetime64[ns]
 4   volume_type     19098 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 746.1+ KB
None


In [17]:
a1_100 = a1.iloc[:100]

In [18]:
def data_frame_to_json( data_frame ):
    result = data_frame.to_json( 'a1_100.json')
    


In [19]:
data_frame_to_json( a1_100 )

In [31]:
column_data_types = a1[a1.columns].dtypes
print( f'{type(column_data_types)} \n{column_data_types}')

<class 'pandas.core.series.Series'> 
path                      object
vin                       object
session_id                object
recording_date    datetime64[ns]
volume_type               object
dtype: object


In [32]:
data = {
    'int_col': [1, 2, 3, 4, 5],
    'float_col': [1.1, 2.2, 3.3, 4.4, 5.5],
    'str_col': ['apple', 'banana', 'cherry', 'date', 'elderberry'],
    'bool_col': [True, False, True, False, True],
    'datetime_col': pd.to_datetime(['2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01']),
    'timedelta_col': [pd.Timedelta(days=1), pd.Timedelta(days=2), pd.Timedelta(days=3), pd.Timedelta(days=4), pd.Timedelta(days=5)],
    'category_col': pd.Categorical(['cat', 'dog', 'dog', 'bird', 'cat']),
    'object_col': [1, 'string', 3.14, True, None],
}

df = pd.DataFrame(data)

# Display the DataFrame and its data types
print(df)
print(df.dtypes)

   int_col  float_col     str_col  bool_col datetime_col timedelta_col  \
0        1        1.1       apple      True   2022-01-01        1 days   
1        2        2.2      banana     False   2022-02-01        2 days   
2        3        3.3      cherry      True   2022-03-01        3 days   
3        4        4.4        date     False   2022-04-01        4 days   
4        5        5.5  elderberry      True   2022-05-01        5 days   

  category_col object_col  
0          cat          1  
1          dog     string  
2          dog       3.14  
3         bird       True  
4          cat       None  
int_col                    int64
float_col                float64
str_col                   object
bool_col                    bool
datetime_col      datetime64[ns]
timedelta_col    timedelta64[ns]
category_col            category
object_col                object
dtype: object


In [33]:
types = {
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'object': 'TEXT',
    'bool': 'BOOL',
    'datetime64': 'DATE',
    'timedelta64': [pd.Timedelta(days=1), pd.Timedelta(days=2), pd.Timedelta(days=3), pd.Timedelta(days=4), pd.Timedelta(days=5)],
    'category': 'TEXT',
    'object_col': [1, 'string', 3.14, True, None],
}

df = pd.DataFrame(types)

# Display the DataFrame and its data types
print(df)
print(df.dtypes)

     int64 float64 object  bool datetime64 timedelta64 category object_col
0  INTEGER   FLOAT   TEXT  BOOL       DATE      1 days     TEXT          1
1  INTEGER   FLOAT   TEXT  BOOL       DATE      2 days     TEXT     string
2  INTEGER   FLOAT   TEXT  BOOL       DATE      3 days     TEXT       3.14
3  INTEGER   FLOAT   TEXT  BOOL       DATE      4 days     TEXT       True
4  INTEGER   FLOAT   TEXT  BOOL       DATE      5 days     TEXT       None
int64                   object
float64                 object
object                  object
bool                    object
datetime64              object
timedelta64    timedelta64[ns]
category                object
object_col              object
dtype: object


In [6]:
class BaseConfig:
    # AIRFLOW_API_ENDPOINT = "{{airflow_api_endpoint}}"
    AIRFLOW_API_ENDPOINT = r'url'

In [7]:
class AirflowDagsMonitoringConfig(BaseConfig):
    DAGS_TO_MONITOR = {
    
        "name1":       {'dag_name ':'dag_name1', 'status': 'success', 'treshold':  24 },
        "name2":        {'dag_name ':'dag_name2', 'status': 'success', 'treshold':  12 },
        "name3": {'dag_name ':'dag_name3', 'status': 'success', 'treshold':  20 },

    }

In [8]:
class AirflowAPIConnection(AirflowDagsMonitoringConfig):
    def __init__(self, username, password, base_url, proxies=None):
        """
        Initialize the AirflowAPI class.

        Parameters:
        - username (str): The username for authentication.
        - password (str): The password for authentication.
        - base_url (str): The base URL of the Airflow REST API.
        - proxies (dict, optional): Proxy settings if needed.
        """
        super().__init__() # Call the constructor of the parent class (AirflowDagsMonitoringConfig)
        self.username = username
        self.password = password
        self.base_url = base_url
        self.proxies = proxies
    
    def get_dag_runs(self, task_id='data_retention'):
        """
        Retrieve DAG runs from the Airflow REST API for a specified task_id.

        Parameters:
        - task_id (str, optional): The ID of the Airflow DAG. Defaults to 'data_retention'.

        Returns:
        - res_j (dict): A dictionary containing the response from the Airflow REST API in JSON format.
        """
        
        auth = HTTPBasicAuth(self.username, self.password)
        connection = {
            "proxies": self.proxies,
            'url': f"{self.base_url}/..../,,,,/..../{task_id}/____",
        }
        with requests.Session() as session:
            session.auth = auth
            session.verify = False
            
            try:
                res = session.get(connection['url'], proxies=connection['proxies'])
            except requests.exceptions.Timeout:
                return "Error: Request timed out", 504
            res_j = res.json()
            return res_j

In [9]:
username= 'username'
password = 'password'
base_url = r'url'

In [10]:
# Example usage:
# Create an instance of AirflowAPIConnection
airflow_conn = AirflowAPIConnection(username, password, base_url)

In [11]:
# Access the configuration data from AirflowDagsMonitoringConfig
config_data = airflow_conn.DAGS_TO_MONITOR

In [ ]:
# class PostgresCredentials:
#     postgres_credentials = {
#         'postgres_username': 'postgres',
#         'postgres_password': 'radek',
#         'postgres_host': 'localhost',
#         'postgres_port': '5432',
#     }

# class PostgresConnection(PostgresCredentials):
#     def __init__(self):
#         super().__init__()
#         self.username = self.postgres_credentials['postgres_username']
#         self.password = self.postgres_credentials['postgres_password']
#         self.host = self.postgres_credentials['postgres_host']
#         self.port = self.postgres_credentials['postgres_port']
    
#     def get_connection_database(self):
#         conn = psycopg2.connect(
#             dbname='postgres',  # Replace with your database name
#             user=self.username,
#             password=self.password,
#             host=self.host,
#             port=self.port
#         )
#         cur = conn.cursor()
#         return conn, cur

#     def read_table(self, cur, table_name):
#         try:
#             cur.execute(f"SELECT * FROM {table_name} LIMIT 5")
#             rows = cur.fetchall()
#             print(f"First 5 rows in the {table_name} table:")
#             for row in rows:
#                 print(row)
#         except Exception as e:
#             print("Error occurred while selecting data:", e)

# # # Create an instance of PostgresConnection
# # postgres_conn = PostgresConnection()

# # # Get the database connection and cursor
# # conn, cur = postgres_conn.get_connection_database()

# # # Read a table (replace 'your_table_name' with the actual table name)
# # postgres_conn.read_table(cur, 'your_table_name')

# # # Close the cursor and connection when done
# # cur.close()
# # conn.close()
    